In [0]:
%run ../source_to_bronze/utils

In [0]:
# MAGIC %run ../source_to_bronze/utils

from pyspark.sql.types import StructType, StructField, StringType, IntegerType

volume_path = "/Volumes/my_catalog/my_schema/my_volume"
bronze_path = f"{volume_path}/bronze"
silver_path = f"{volume_path}/silver/employee_info/dim_employee"

# Define custom schema for employee
employee_schema = StructType([
    StructField("EmployeeID", IntegerType()),
    StructField("EmployeeName", StringType()),
    StructField("Department", StringType()),
    StructField("Country", StringType()),
    StructField("Salary", IntegerType()),
    StructField("Age", IntegerType())
])

# Read using custom schema
df = spark.read.schema(employee_schema).option("header", True).csv(f"{bronze_path}/employee")

# Transform
df = rename_columns_to_snake_case(df)
df = add_load_date(df)

# Write to Silver (as Delta)
df.write.format("delta").mode("overwrite").save(silver_path)




In [0]:
df = spark.read.format("delta").load(silver_path)
df.display(truncate=False)
df.printSchema()
